Tavola 8.6  - Abbonamenti alla radio e alla televisione italiana – Anni 1936-2014 

In [7]:
import sqlite3

import pandas as pd
import os
import plotly.express as px
import sqlite3 as db

Qui vengono caricati i path della cartella dedicata ai Datasets e del percorso specifico della serie Istat

In [8]:
df_dir = os.path.join(os.getcwd(), 'Datasets')
df_path = (os.path.join(df_dir, 'Tavola_8.6.xlsx'))

La struttura della tabella non è pulita, dunque si è scelto di saltare le prime cinque righe (contenenti un'intestazione dell'Istat) e la sesta, in quanto quest'ultima è aggregata a quella successiva per la riga di intestazione della tabella. Inoltre saltiamo le ultime righe contenente solo una legenda.

In [10]:
df1 = pd.read_excel(df_path, sheet_name='Tavola 8.6', skiprows=6, skipfooter=5) # sheet_name indica il foglio specifico da cui voglio creare il df dal file
df2 = pd.read_excel(df_path, sheet_name='Tavola 8.6 (segue)', skiprows=6 , skipfooter=6) # il parametro skiprows viene speficicato per dedicere quante righe saltare al file della creazione del df

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Program Files\\JetBrains\\PyCharm 2024.1.4\\jbr\\bin\\Datasets\\Tavola_8.6.xlsx'

In [ ]:
df1

In [ ]:
df2

Notiamo che l'aggregazione delle righe sul file excel sta generando confusione per pandas, dunque rinominiamo manualmente l'header della tabella. Inoltre notiamo che la riga 0 è vuota, quindi possiamo eliminarla. La stessa cosa vale per la terza colonna, che è vuota e non rappresenta nulla (su entrambi gli sheet).

In [ ]:
columns = [
    "Anno", 
    "Radio (a)",
    "TV - Uso privato", 
    "TV - Speciali (b)", 
    "TV - Totale", 
    "Abbonamenti TV per 1000 abitanti",
    "Abbonamenti TV per 100 famiglie"
]

In [ ]:
df1.drop('Unnamed: 2', axis=1, inplace=True) # axis = 1 in questo caso fa in modo di eliminare una colonna, inplace settato a true fa operare sullo stesso df
df2.drop('Unnamed: 2', axis=1, inplace=True) 
df1.drop(0, inplace=True)
df2.drop(0, inplace=True)

df1.columns=columns
df2.columns=columns

Notiamo che una parte degli anni sembra essere un float, quindi specifichiamo il valore a int

In [ ]:
df1["Anno"] = df1["Anno"].astype(int)

Concateniamo ora i due dataframe derivanti dai due fogli del file

In [ ]:
df = pd.concat([df1, df2], ignore_index=True) # dal momento che l'indice riparte da 0 nel secondo foglio, settiamo ignore_index a true

In [ ]:
df

La struttura del df è parzialmente pulita, resta solo da rendere come index la colonna anno e pulire l'anno 2002

In [ ]:
df.loc[df["Anno"] == "2002 (c)", "Anno"] = 2002 # il primo è l'indice della riga, il secondo della colonna

In [ ]:
df.set_index("Anno", inplace=True)

Analizzando la colonna degli ascolti radio si nota che molti valori sono sporchi (rappresentati con "..."). Per pulire il df, rimpiazziamo questi valori con NaN.

In [ ]:
df.replace("….", pd.NA, inplace=True)

Ciononostante la colonna non è ancora numerica, quindi dobbiamo convertirla

In [ ]:
df["Radio (a)"] = pd.to_numeric(df["Radio (a)"], errors="coerce")

Ammettiamo di voler lavorare solo sugli ascolti radio (li prendiamo dal 1936 al 1989 in quanto in seguito i dati non sono disponibili)

In [ ]:
df_radio = df.loc[(df.index >= 1936) & (df.index <= 1989), ['Radio (a)']].copy()


Nota bene, questo codice invece non avrebbe funzionato in quanto "Anno" è un index

df_radio = df.loc[
    (df["Anno"] >= 1936) & (df["Anno"] <= 1989),  
    ["Anno", "Radio (a)"]
].copy()

Possiamo provare ad interpolare i dati mancanti (dal momento che ce ne sono pochi)

In [ ]:
df_radio["Radio (a)"] = df_radio["Radio (a)"].interpolate(method="linear")

In [ ]:
df_radio.plot(kind='line') 

Con plotly express

In [ ]:
fig = px.line(df_radio, x=df_radio.index, y="Radio (a)")
fig.show()

Proviamo a creare un database per inserirci i dati dal dataframe. Creiamo inizialmente una cartella Database.

In [ ]:
db_path = (os.path.join(os.getcwd(), 'Database'))

Dopodichè creiamo una connessione al database, se non esiste il file (in questo caso 'database.db') viene creato.

In [ ]:
conn = db.connect(os.path.join(db_path, 'database.db'))

Creiamo la tabella ascolti_radio sul db, specificandone il name e chiamando il metodo .to_sql sul df

Creiamo il cursore

In [ ]:
cursor = conn.cursor()

In [ ]:
cursor.execute("DROP TABLE IF EXISTS ascolti_radio")

In [ ]:
cursor.execute("""CREATE TABLE IF NOT EXISTS ascolti_radio (
Anno INTEGER PRIMARY KEY,
"Radio (a)" DOUBLE,
"TV - Uso privato" DOUBLE
)""")

In [ ]:
df_radio.to_sql(name='ascolti_radio', con=conn, if_exists='append')

Eseguiamo la query

In [ ]:
cursor.execute("SELECT * FROM ascolti_radio")

Salviamone i risultati nella variabile rows e dopodichè stampiamo i risultati

In [ ]:
rows = cursor.fetchall()

In [ ]:
for row in rows:
    print(row)

Fare riferimento al modulo main per la creazione delle API

Tornando alla dataviz, cerchiamo ora di concatenare (anche se si poteva fare solo un sottodf di quello iniziale) anche gli ascolti TV

Puliamo inizialmente gli ascolti TV privati

In [ ]:
df.replace("-", pd.NA, inplace=True)

In [ ]:
df["TV - Uso privato"] = pd.to_numeric(df["TV - Uso privato"], errors="coerce")

In [ ]:
df_tv_priv = df.loc[(df.index >= 1936) & (df.index <= 1989), ['TV - Uso privato']].copy()

In [ ]:
df_tv_priv

Dobbiamo affiancare le colonne, dunque utilizziamo axis=1

In [ ]:
df_radio_tv_priv = pd.concat([df_radio, df_tv_priv], axis=1)

In [ ]:
df_radio_tv_priv

Svuotiamo la tabella (nota, non esiste truncate)

In [ ]:
cursor.execute("DELETE FROM ascolti_radio")

In [ ]:
df_radio_tv_priv.to_sql(name='ascolti_radio', con=conn, if_exists='append')

Creiamo ora una nuova figura passando le due serie sull'asse Y

In [ ]:
fig2 = px.line(df_radio_tv_priv, x=df_radio_tv_priv.index, y=["Radio (a)", "TV - Uso privato"])
fig2.show()


In [ ]:
df_radio_tv_priv.to_csv(os.path.join(os.getcwd(), 'Outputs', 'ascolti_radio_tv_priv.csv'))